In [9]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
import numpy as np
import json

In [10]:
def generate_seq(img_input):
    
    if img_input.shape != (1, 512):
        img_input = img_input.reshape(1, 512)

    
    assert(img_input.shape == (1, 512))
    stop_condition = False
    decoded_sentence = []
    target_seq = np.array([token2idx['<bos>']]).reshape(1, 1)
    states_value = encoder_model.predict(img_input)

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))
        sampled_char = idx2token[sampled_token_index]
        decoded_sentence += [sampled_char]
        if (sampled_char == '<eos>' or len(decoded_sentence) > 30):
            stop_condition = True
        target_seq = np.array([sampled_token_index]).reshape(1, 1)
        states_value = [h, c]

    return ' '.join(decoded_sentence[:-1])

In [11]:
VGG16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

def get_captions(img_path):   
    #img_path = 'data/Arnav_Hankyu_Pulkit2.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = VGG16_model.predict(x)
    return generate_seq(features)

In [12]:
from caption_utils import *

In [13]:
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()
train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

C:\Users\hankyu\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:1541: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'img_embedding_1/Relu:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'img_embedding_1/Relu:0' shape=(?, 300) dtype=float32>]
  ' Found: ' + str(self.outputs))
C:\Users\hankyu\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


To use `corpus_bleu`, number of references per image (5) should match the number of candidates (5).

In [14]:
from nltk.translate.bleu_score import corpus_bleu

In [15]:
bleu_scores = np.zeros(len(test_fns_list))
for i, filename in enumerate(test_fns_list):
    if i%100 == 0:
        print(i, "images processed")
    references = []
    for caption in test_captions_raw[filename]:
        references.append(caption[:-1].split())
    candidates = [get_captions("data/Flicker8k_Dataset/"+filename).split()] * 5
    bleu_scores[i] = corpus_bleu(references, candidates)

0 images processed


C:\Users\hankyu\Anaconda3\envs\tensorflow\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


100 images processed
200 images processed
300 images processed
400 images processed
500 images processed
600 images processed
700 images processed
800 images processed
900 images processed


In [16]:
print("Bleu Score: ", bleu_scores.mean())

Bleu Score:  0.5399454288695043
